<a href="https://colab.research.google.com/github/kairamilanifitria/NLP-Projects/blob/main/Project%202%20Text%20Summarization/Fix/3_Modelling_BERT_Extractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extractive Model

In [1]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [3]:
# Load the dataset
train_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/train_df.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/dev_df.csv')

# Basic overview
print(train_df.shape)
print(train_df.info())
print(train_df.isnull().sum())

print(dev_df.shape)
print(dev_df.info())
print(dev_df.isnull().sum())


(29842, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29842 entries, 0 to 29841
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  29842 non-null  int64 
 1   original_text               29842 non-null  object
 2   abstractive_summary         29842 non-null  object
 3   extractive_summary          29842 non-null  object
 4   original_text_length        29842 non-null  int64 
 5   abstractive_summary_length  29842 non-null  int64 
 6   extractive_summary_length   29842 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.6+ MB
None
Unnamed: 0                    0
original_text                 0
abstractive_summary           0
extractive_summary            0
original_text_length          0
abstractive_summary_length    0
extractive_summary_length     0
dtype: int64
(3000, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
D

In [4]:
columns_to_delete = ['Unnamed: 0', 'abstractive_summary', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']

# Delete the specified columns from the DataFrame
train_df = train_df.drop(columns=columns_to_delete)
dev_df = dev_df.drop(columns=columns_to_delete)


In [5]:
train_df.iloc[1]

,1
original_text,liputan6 com jakarta karyawan pt angkasa pura ...
extractive_summary,liputan6 com jakarta karyawan pt angkasa pura ...


In [6]:
dev_df.iloc[1]

,1
original_text,liputan6 com jakarta peruntungan ahmad taufik ...
extractive_summary,seperti tak mau ketinggalan presiden abdurrahm...


In [7]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')


Using device: cuda


In [8]:
# Load pre-trained BERT model for text summarization (extractive)
model_name = "cahya/bert2bert-indonesian-summarization"
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [9]:
# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [10]:
from transformers import TrainingArguments, Trainer

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

def preprocess_function_extractive(examples):

    extractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["extractive_summary"]
    ]  # List comprehension for handling list summaries

    # Tokenize the text input
    model_inputs = tokenizer(
        examples["original_text"], max_length=512, truncation=True, padding="max_length"
    )

    # Tokenize the combined summary (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            extractive_summary, max_length=256, truncation=True, padding="max_length"
        )

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
# Apply preprocessing to the train and dev datasets
tokenized_train_datasets = train_dataset.map(preprocess_function_extractive, batched=True)
tokenized_dev_datasets = dev_dataset.map(preprocess_function_extractive, batched=True)

Map:   0%|          | 0/29842 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    fp16=True,
    save_steps=500,           # Save the model every 500 steps
    eval_steps=500,           # Evaluate every 500 steps
    logging_dir="./logs",     # Directory for logs
    logging_steps=100,        # Log every 100 steps
)


# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,  # Use tokenized train dataset
    eval_dataset=tokenized_dev_datasets,     # Use tokenized dev dataset for validation
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [14]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss
1,0.047500,0.036906
2,0.031900,0.032625
3,0.022400,0.032483


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Some

TrainOutput(global_step=11193, training_loss=0.047572079883187936, metrics={'train_runtime': 8134.9817, 'train_samples_per_second': 11.005, 'train_steps_per_second': 1.376, 'total_flos': 5.49204982848553e+16, 'train_loss': 0.047572079883187936, 'epoch': 3.0})

In [15]:
import os

# Define the path in Google Drive where you want to save the model
output_dir = '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/BERT:Extractive'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


Model saved to /content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/BERT:Extractive


In [16]:
test_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/test_df.csv')

In [17]:
columns_to_delete = ['Unnamed: 0', 'abstractive_summary', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']
# Delete the specified columns from the DataFrame
test_df = test_df.drop(columns=columns_to_delete)
test_df.head()

,original_text,extractive_summary
0,liputan6 com jakarta ratusan mahasiswa se jabo...,liputan6 com jakarta ratusan mahasiswa se jabo...
1,liputan6 com palembang puluhan orang perwakila...,mereka menuntut perusahaan mengembalikan tanah...
2,liputan6 com jakarta nangroe aceh darussalam d...,liputan6 com jakarta nangroe aceh darussalam d...
3,liputan6 com jakarta tidak ada perbedaan peraw...,perbedaan hanya pada pelayanan kepada para pen...
4,liputan6 com jakarta menteri luar negeri demis...,pernyataan itu disampaikan alwi seusai bertemu...


In [18]:
test_df.shape

(2991, 2)

In [19]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_extractive_summaries_for_df(df):
    model_path = "/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/BERT:Extractive"
    summarizer =EncoderDecoderModel.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = BertTokenizer.from_pretrained(model_path)

    # Lists to store both types of summaries
    extractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            extractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            extractive_summaries.append(extractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            extractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'extractive_summary': extractive_summaries
    })

# Assuming test_df_subset is already defined
extractive_summary_BERT_df = generate_extractive_summaries_for_df(test_df)

extractive_summary_BERT_df


Generating Summaries: 100%|██████████| 2991/2991 [51:47<00:00,  1.04s/it]


,extractive_summary
0,dalam aksinya mereka mengaku kecewa sebab hing...
1,mereka menuntut perusahaan mengembalikan tanah...
2,liputan6 com jakarta nangroe aceh darussalam d...
3,liputan6 com jakarta tidak ada perbedaan peraw...
4,menurut alwi sejak dulu pemerintah konsisten m...
...,...
2986,isi ruu tersebut diharapkan dapat menciptakan ...
2987,seluruh penjualan buku tersebut akan disumbang...
2988,nuansa pembauran antara masyarakat tionghoa da...
2989,liputan6 com jakarta kepolisian daerah metro j...


In [20]:
from google.colab import files

csv_file_path = 'extractive_summary_BERT_df.csv'
extractive_summary_BERT_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ROUGE CALCULATION**

In [21]:
!pip install datasets transformers torch accelerate rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1a372d467c64f8da5ff2bfadb5003ec01d915ffac382751f05233bf792f8b801
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [22]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer
from transformers import pipeline

In [23]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [24]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = extractive_summary_BERT_df['extractive_summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"Average ROUGE-1 fine-tuned BERT model extractive summary")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1 fine-tuned BERT model extractive summary
Average ROUGE-1: 0.48416238084327734
Average ROUGE-2: 0.399254182843347
Average ROUGE-L: 0.4478177775648456


comparison to raw model

In [25]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_extractive_summaries_for_df(df):
    model_path = "cahya/bert2bert-indonesian-summarization"
    summarizer =EncoderDecoderModel.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = BertTokenizer.from_pretrained(model_path)

    # Lists to store both types of summaries
    extractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            extractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            extractive_summaries.append(extractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            extractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'extractive_summary': extractive_summaries
    })

# Assuming test_df_subset is already defined
extractive_summary_BERT_raw = generate_extractive_summaries_for_df(test_df)

extractive_summary_BERT_raw


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating Summaries: 100%|██████████| 2991/2991 [40:57<00:00,  1.22it/s]


,extractive_summary
0,ratusan mahasiswa se jabotabek memperingati ti...
1,puluhan warga desa kota baru dan desa pagar gu...
2,ksad jenderal tni endriartono sutarto berharap...
3,tidak ada perbedaan perawatan antara kereta ap...
4,menlu demisioner alwi shihab menegaskan posisi...
...,...
2986,dpr dan pemerintah pusat didesak untuk segera ...
2987,buku kumpulan lukisan karya almarhum hendra gu...
2988,cerita percintaan antara sami un dan ing tay d...
2989,operasi ketupat lebaran yang bertujuan untuk m...


In [26]:
from google.colab import files

csv_file_path = 'extractive_summary_BERT_raw.csv'
extractive_summary_BERT_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = extractive_summary_BERT_raw['extractive_summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"Average ROUGE-1 raw BERT model extractive summary")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1 raw BERT model extractive summary
Average ROUGE-1: 0.46801917505179136
Average ROUGE-2: 0.36160433473543696
Average ROUGE-L: 0.42796482420016035
